In [14]:
import os
import glob

import pandas as pd

In [21]:
DATA_DIR = '../data'
WEIGHT_MIN = 1
WEIGHT_MAX = 100

In [26]:
def get_weights(group):
    min_count = group['Count'].min()
    max_count = group['Count'].max()
    print(min_count, max_count, WEIGHT_MIN + (group['Count'] - min_count) * (WEIGHT_MAX - 1) / (max_count - min_count))
    
    group['Weight'] = WEIGHT_MIN + (group['Count'] - min_count) * (WEIGHT_MAX - 1) / (max_count - min_count)
    return group

In [27]:
all_dfs = []

csv_files = glob.glob(
    os.path.join(f"{DATA_DIR}/tags", '*.csv')
)
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    all_dfs.append(df)
    
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df = combined_df.groupby('Tag', as_index=False)['Count'].sum()
combined_df = combined_df.sort_values(by='Count', ascending=False)

combined_df[['Key', 'Value']] = combined_df['Tag'].str.extract(r"\('([^']*)', '([^']*)'\)")

combined_df = combined_df[
    combined_df['Key'].isin([
        'highway', 
        'surface',
        'footway',
        'surface',
        'oneway',
        'lit',
        'name',
        'crossing',
    ])
]

combined_df = combined_df.groupby('Key').apply(get_weights)

output_dir = f"{DATA_DIR}/weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

combined_df.to_csv(f"{output_dir}/weights.csv", index=False)

78 671 32    100.000000
31     92.821248
30     91.652614
33     22.703204
34      1.000000
Name: Count, dtype: float64
2241 4337 51    100.0
50      1.0
Name: Count, dtype: float64
8 12556 64    100.000000
61     63.399665
56     55.478403
66      7.445888
59      7.106631
62      3.721948
60      2.483264
65      2.325470
58      1.528610
57      1.449713
63      1.370816
67      1.142015
69      1.142015
68      1.000000
Name: Count, dtype: float64
467 467 87   NaN
Name: Count, dtype: float64
1 543 301    100.000000
346     65.295203
350     53.422509
351     45.933579
352     32.234317
          ...    
136      1.000000
343      1.000000
328      1.000000
144      1.000000
336      1.000000
Name: Count, Length: 243, dtype: float64
253 2158 456    100.0
455      1.0
Name: Count, dtype: float64
11 5835 476    100.000000
477      9.159341
480      5.181662
478      4.365728
481      1.169986
479      1.000000
Name: Count, dtype: float64
